In [12]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup

In [62]:
def func_get_text(cell):
    return cell.get_text(strip=True)

In [ ]:
def process_radiation_search(download_path):
    html_file = open(download_path, "r")
    soup = BeautifulSoup(html_file.read(), 'lxml')
    table = soup.find_all("table")[0]
    gammas = table.find_all("tr")
    init_dict = {"E_tab": [0], "Ig": [0], "Decay_Mode": ["b-"], "half_life": ["3 m"], "Isotope": ["Tu"]}
    gamma_df = pd.DataFrame(init_dict)

    for gamma_row in gammas:
        vals = list(map(func_get_text, row.find_all("td")))
        gamma_df.append({"E_tab": vals[0], "Ig": vals[1], "Decay_Mode": vals[2], "half_life": vals[3], "Isotope": vals[4]}, ignore_index=True)
    return gamma_df

In [10]:
# http://nucleardata.nuclear.lu.se/toi/Gamma.asp?sql=&Min=58&Max=60&HlifeMin=2&tMinStr=2+s&HlifeMax=1000&tMaxStr=1000+s
def find_peaks(energy, fwhm, t_min, t_max):
    e_min = energy - fwhm
    e_max = energy + fwhm
    request_url = f"http://nucleardata.nuclear.lu.se/toi/Gamma.asp?sql=&Min={e_min}&Max={e_max}&HlifeMin={t_min}&tMinStr={t_min}+s&HlifeMax={t_max}&tMaxStr={t_max}+s"
    output_file = f"downloads/searchE{energy}pm{fwhm}tMin{t_min}tMax{t_max}.html"
    urllib.request.urlretrieve (request_url, output_file)
    print(f"File {output_file} downloaded.")
    gamma_df = process_radiation_search(output_file)
    gamma_df["Energy"] = energy
    gamma_df["FWHM"] = fwhm
    gamma_df["half_life_searched"] = f"{t_min} - {t_max}"
    gamma_df = gamma_df.drop(rows=[0])
    return gamma_df

In [ ]:
all_df = 
#found_peaks = parsed_df.apply(find_peaks, args=)